In [3]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yfinance as yf
import gurobipy as gp

print(cp.installed_solvers())

['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


In [4]:
with gp.Env(empty=True) as env:
    env.setParam("OutputFlag", 0)
    env.start()

In [170]:
def optimize_portfolio_with_risk(stocks, returns, cov_matrix, min_return, max_stocks):
    """
    Select a certain amount of stocks to minimize risk subject to a minimum expected return.

    Parameters:
    - stocks: List of stock symbols.
    - returns: Array of expected returns for each stock.
    - cov_matrix: Covariance matrix of stock returns.
    - min_return: Minimum expected return for the portfolio.
    - max_stocks: Maximum number of stocks to select.

    Returns:
    - A tuple of the optimized portfolio, its expected return, and its risk (standard deviation).
    """

    n = len(stocks) # Number of stocks
    # Decision variables
    x = cp.Variable(n, boolean=True) # Binary variables for stock selection
    w = cp.Variable(n) # Continuous variables for weights

    # Objective: Minimize portfolio risk (standard deviation)
    risk = cp.quad_form(w, cov_matrix)
    objective = cp.Minimize(risk)

    # Constraints
    constraints = [
        cp.sum(w) == 1, # Sum of weights is 1
        w >= 0, # No short selling
        cp.sum(x) == max_stocks,
        w <= x, # Select exactly max_stocks stocks
        cp.sum(cp.multiply(w, returns)) >= min_return # Minimum expected return
    ]

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.GUROBI)

    # Check if a valid solution exists
    if prob.status == cp.OPTIMAL or prob.status == cp.OPTIMAL_INACCURATE:
        selected_stocks = [stocks[i] for i in range(n) if round(x.value[i]) == 1]
        portfolio_return = sum(w.value[i] * returns[i] for i in range(n))
        portfolio_risk = np.sqrt(risk.value)
        return selected_stocks, portfolio_return, portfolio_risk, w.value
    else:
        raise Exception("No optimal solution found.")

In [171]:
"""
IJH: US Mid cap
IWM: Russell 2000, small cap US stocks
SPY: S&P 500 ETF, to capture 
EEM: Emerging Markets ETF
EWJ: Japan ETF
TIP: Tracks TIPS (Treasury inflation-protected securities)
DBO: Tracks WTI
FXE: Tracks the price of Euro
CEW: Tracks the price of EM currencies against USD
USCI: Tracks 14 commodity futures contracts from 5 different sectors
"""


tickers = ['IJH', 'IWM', "SPY", "EEM", "EWJ", "TIP", "DBO", "FXE","CEW","USCI"]

In [172]:
# REPLACE WITH REAL DATA
stocks = ['1', '2', '3', '4', '5']
expected_returns = np.array([0.1, 0.12, 0.15, 0.08, 0.2]) # This should be actual data
cov_matrix = np.random.rand(5, 5) # This should be an actual covariance matrix
cov_matrix = (cov_matrix + cov_matrix.T) / 2 # Ensure the matrix is symmetric
np.fill_diagonal(cov_matrix, cov_matrix.diagonal() * 10) # Increase variance on the diagonal for realism
min_return = 0.005
max_stocks = 3

selected_stocks, portfolio_return, portfolio_risk, weight = optimize_portfolio_with_risk(stocks, expected_returns, cov_matrix, min_return, max_stocks)
print(f"Selected Stocks: {selected_stocks}")
print(f"Portfolio Expected Return: {portfolio_return}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")
print(weight)

Selected Stocks: ['2', '4', '5']
Portfolio Expected Return: 0.1473871206562924
Portfolio Risk (Std. Dev.): 1.0213091161280368
[0.         0.14364836 0.         0.34267509 0.51367655]


In [178]:
#Import ETFs
data = yf.download(tickers, start="2014-01-01", end="2024-01-01")['Adj Close'].dropna()

# Calculate daily returns
daily_returns = data.pct_change() * 100

# Calculate expected returns (mean of daily returns)
expected_returns = daily_returns.mean() 

# Calculate covariance matrix of returns
covariance_matrix = daily_returns.cov()

# Print the expected returns and covariance matrix
print("Expected Returns:\n", expected_returns)
print("Covariance Matrix:\n", covariance_matrix)


[*********************100%%**********************]  10 of 10 completed
Expected Returns:
 CEW     0.000075
DBO    -0.001925
EEM     0.017282
EWJ     0.023723
FXE    -0.009349
IJH     0.068501
IWM     0.037555
SPY     0.051286
TIP     0.009406
USCI    0.004891
dtype: float64
Covariance Matrix:
            CEW       DBO       EEM       EWJ       FXE       IJH       IWM  \
CEW   0.217216  0.303230  0.399663  0.225804  0.105056  0.276011  0.289186   
DBO   0.303230  4.179543  0.940496  0.624117  0.081580  0.911345  0.955201   
EEM   0.399663  0.940496  1.699286  0.974808  0.136573  1.212222  1.271016   
EWJ   0.225804  0.624117  0.974808  1.132521  0.068457  0.974070  1.023971   
FXE   0.105056  0.081580  0.136573  0.068457  0.252999  0.067534  0.070200   
IJH   0.276011  0.911345  1.212222  0.974070  0.067534  1.685892  1.721975   
IWM   0.289186  0.955201  1.271016  1.023971  0.070200  1.721975  1.941386   
SPY   0.234131  0.746181  1.092703  0.876825  0.053344  1.292370  1.346129   
TIP

In [179]:
# Construct large portfolio
min_return = 0.001
max_stocks = 10

selected_stocks, portfolio_return, portfolio_risk, w= optimize_portfolio_with_risk(tickers, expected_returns, covariance_matrix, min_return, max_stocks)
print(f"Selected Stocks: {selected_stocks}")
print(f"Portfolio Expected Return: {portfolio_return}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")
print(w)

Selected Stocks: ['IJH', 'IWM', 'SPY', 'EEM', 'EWJ', 'TIP', 'DBO', 'FXE', 'CEW', 'USCI']
Portfolio Expected Return: 0.004663032527931856
Portfolio Risk (Std. Dev.): 0.29985674485509684
[2.44827926e-01 7.07267665e-13 5.92451382e-13 1.92925570e-03
 1.67645228e-01 2.25704931e-12 1.49324406e-12 1.62387239e-02
 5.64576381e-01 4.78248512e-03]


In [180]:
# Construct medium portfolio
min_return = 0.0001
max_stocks = 5

selected_stocks, portfolio_return, portfolio_risk, weights = optimize_portfolio_with_risk(tickers, expected_returns, covariance_matrix, min_return, max_stocks)
print(f"Selected Stocks: {selected_stocks}")
print(f"Portfolio Expected Return: {portfolio_return}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")
print(f"Sharpe Ratio: {portfolio_return/portfolio_risk}")
print(weights)

Selected Stocks: ['IJH', 'EWJ', 'FXE', 'CEW', 'USCI']
Portfolio Expected Return: 0.00468205193619499
Portfolio Risk (Std. Dev.): 0.29985973663745563
Sharpe Ratio: 0.01561414009329238
[0.24546068 0.         0.         0.         0.16763003 0.
 0.         0.01749241 0.56458006 0.00483682]


In [181]:
# Construct medium portfolio
min_return = 0.0001
max_stocks = 3

selected_stocks, portfolio_return, portfolio_risk, weights = optimize_portfolio_with_risk(tickers, expected_returns, covariance_matrix, min_return, max_stocks)
print(f"Selected Stocks: {selected_stocks}")
print(f"Portfolio Expected Return: {portfolio_return}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")
print(f"Sharpe Ratio: {portfolio_return/portfolio_risk}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")
print(weights)

Selected Stocks: ['IJH', 'EWJ', 'CEW']
Portfolio Expected Return: 0.0038121483349169297
Portfolio Risk (Std. Dev.): 0.30042443760355286
Sharpe Ratio: 0.012689208525531235
Portfolio Risk (Std. Dev.): 0.30042443760355286
[0.27108934 0.         0.         0.         0.16336496 0.
 0.         0.         0.5655457  0.        ]
